# Download sentinel 2 images 

In [2]:
from scripts.download_s2_pc_by_tile import run as run_download

run_download(
    outdir="data/sentinel",
    aoi_path="data/aoi/CMDA.shp",
    year=2025,
)

/Users/athithiyan/anaconda3/envs/sentinel2/lib/python3.10/site-packages/pystac_client/item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(
2026-01-27 11:39:46,145 INFO: Found 175 Sentinel-2 items
2026-01-27 11:39:50,378 INFO: [T44PMV January] downloaded SCL


KeyboardInterrupt: 

# Calculating mean indices [NDVI, NDBI,BSI,NDWI]

In [1]:
from scripts.mean_indices import run as run_indices

run_indices(
    tiles=["T44PLU"],
    years=[2025],
    indices=["NDVI", "NDBI", "BSI", "MNDWI", "SAVI", "IBI"],
    aggregate_mean=True,
    aggregate_median=True,
    overwrite_monthly=False,
)

# indices=("NDVI","NDBI","BSI","MNDWI","SAVI","IBI")

2026-01-27 12:21:44,942 INFO: [2025 January] indices exist → skipped
2026-01-27 12:21:44,984 INFO: [2025 February] indices exist → skipped
2026-01-27 12:21:45,119 INFO: [2025 March] indices exist → skipped
2026-01-27 12:21:45,156 INFO: [2025 April] indices exist → skipped
2026-01-27 12:21:45,164 INFO: [2025 May] indices exist → skipped
2026-01-27 12:21:45,170 INFO: [2025 June] indices exist → skipped
2026-01-27 12:21:45,181 INFO: [2025 July] indices exist → skipped
2026-01-27 12:21:45,199 INFO: [2025 August] indices exist → skipped
2026-01-27 12:21:45,215 INFO: [2025 September] indices exist → skipped
2026-01-27 12:21:45,222 INFO: [2025 October] indices exist → skipped
2026-01-27 12:21:45,254 INFO: [2025 November] indices exist → skipped
2026-01-27 12:21:45,271 INFO: [2025 December] indices exist → skipped


[########################################] | 100% Completed | 183.71 s
[##########################              ] | 65% Completed | 117.00 s

/Users/athithiyan/anaconda3/envs/sentinel2/lib/python3.10/site-packages/dask/_task_spec.py:767: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)


[########################################] | 100% Completed | 174.53 s


2026-01-27 12:29:37,024 INFO: [T44PLU 2025] aggregated NDVI


...

2026-01-27 12:58:24,151 INFO: [T44PLU 2025] aggregated SAVI


[########################################] | 100% Completed | 182.21 s
[##########################              ] | 65% Completed | 128.05 s

/Users/athithiyan/anaconda3/envs/sentinel2/lib/python3.10/site-packages/dask/_task_spec.py:767: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)


[########################################] | 100% Completed | 188.29 s


2026-01-27 13:05:39,612 INFO: [T44PLU 2025] aggregated IBI


# Model and predictions

In [1]:
from scripts.train_and_predict_builtup import run as run_builtup

feature_files = [
#     "{base}_MEAN_NDVI.tif",
    "{base}_MEDIAN_NDVI.tif",
#     "{base}_MEAN_NDBI.tif",
    "{base}_MEDIAN_NDBI.tif",
#     "{base}_MEAN_MNDWI.tif",
    "{base}_MEDIAN_MNDWI.tif",
#     "{base}_MEAN_SAVI.tif",
    "{base}_MEDIAN_SAVI.tif",
#     "{base}_MEAN_IBI.tif",
    "{base}_MEDIAN_IBI.tif",
]

index_thresholds = {
    "NDVI":  (-0.1, 0.5),
    "MNDWI": (-0.6, -0.05),
    "NDBI":  (0.05, 1.0),
    "IBI":   (0.0, 1.0),
}

clf = run_builtup(
    root="data/sentinel",
    year="2025",
    train_vector="data/training/CMDA_overall.shp",
    class_col="class",
    tiles=["T44PLV"],
    feature_files=feature_files,
    index_thresholds=index_thresholds,
    max_samples_per_poly=100,
    n_trees=100,
    prob_threshold=0.7,
    out_model="output/model/builtup_rf_custom_200.joblib",
)

2026-01-28 11:47:12,205 INFO: Extracting training samples → T44PLV
2026-01-28 11:47:12,542 INFO: [T44PLV] Class 1 → 1 samples
2026-01-28 11:47:13,790 INFO: [T44PLV] Class 1 → 92 samples
2026-01-28 11:47:14,461 INFO: [T44PLV] Class 1 → 2 samples
2026-01-28 11:47:14,858 INFO: [T44PLV] Class 1 → 4 samples
2026-01-28 11:47:15,279 INFO: [T44PLV] Class 1 → 7 samples
2026-01-28 11:47:15,920 INFO: [T44PLV] Class 0 → 48 samples
2026-01-28 11:47:16,251 INFO: [T44PLV] Class 0 → 1 samples
2026-01-28 11:47:17,115 INFO: [T44PLV] Class 0 → 1 samples
2026-01-28 11:47:17,118 INFO: [T44PLV] Class 0 → 1 samples
2026-01-28 11:47:17,945 INFO: [T44PLV] Class 0 → 1 samples
2026-01-28 11:47:18,038 INFO: [T44PLV] Class 0 → 3 samples
2026-01-28 11:47:18,841 INFO: [T44PLV] Class 0 → 2 samples
2026-01-28 11:47:25,637 INFO: [T44PLV] Class 0 → 10 samples
2026-01-28 11:47:26,480 INFO: [T44PLV] Class 0 → 8 samples
2026-01-28 11:47:26,850 INFO: [T44PLV] Class 0 → 1 samples
2026-01-28 11:47:28,738 INFO: [T44PLV] Class 

In [1]:
from scripts.build_dl_stack import run as run_dl_stack

run_dl_stack(
    root="data/sentinel",
    tiles=["T44PLU"],
    year=2025,
    month="October"
)

2026-01-28 16:59:43,055 INFO: [T44PLU] DL stack built → S2_DL_STACK_2025_October_WITH_INDICES.tif


In [5]:
from scripts.make_patches_dl import run as run_make_patches

run_make_patches(
    root="data/sentinel",
    out_dir="data/patches_osm_oct_2025",
    tiles=["T44PLU"],
    MONTH="October",
    year=2025
)

2026-01-28 17:34:36,796 WARNING: [T44PLU] stack missing


In [1]:
from scripts.deep_learning import run as run_dl_model

run_dl_model(
    patch_img="data/patches/images",
    patch_msk="data/patches/masks",
    sentinel_root="data/sentinel",
    tiles=["T44PMV"],
    year=2025,
    epochs=20,
    batch_size=4,
    threshold=0.6,
)

/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, 

Using device: mps

--- STEP 1: TRAINING ---


Epoch 1/20:   0%|                                                                                                                                                                                                                                        | 0/97 [00:00<?, ?it/s]/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anacon

Train loss: 1.2240465455448504
Val loss: 0.923682096004486


Epoch 2/20:   0%|                                                                                                                                                                                                                                        | 0/97 [00:00<?, ?it/s]/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anacon

...

Train loss: 0.23248623733975224
Val loss: 0.2147962399572134


Epoch 19/20:   0%|                                                                                                                                                                                                                                       | 0/97 [00:00<?, ?it/s]/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anacon

Train loss: 0.22084756879155168
Val loss: 0.22199649035930633


Epoch 20/20:   0%|                                                                                                                                                                                                                                       | 0/97 [00:00<?, ?it/s]/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anaconda3/envs/lila/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/athithiyan/anacon

Train loss: 0.2338523597830964
Val loss: 0.17758943930268287

--- STEP 2: PREDICTION ---

🎉 TRAINING + PREDICTION COMPLETED
